In [20]:
# import tensorflow to see if GPU is connected
import tensorflow as tf
import tensorflow_datasets as tfds
import os
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
data_dir = os.getcwd()
tf.config.list_physical_devices('GPU')

TensorFlow version: 2.7.0
Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [21]:
# loading the MINIST data sets
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    data_dir=data_dir,
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [22]:
%%time
# runing the model to test the performance
@tf.autograph.experimental.do_not_convert
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

Epoch 1/12


2022-01-09 18:51:57.778998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


468/469 [============================>.] - ETA: 0s - loss: 0.1606 - accuracy: 0.9524

2022-01-09 18:52:05.865770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 9s 16ms/step - loss: 0.1604 - accuracy: 0.9524 - val_loss: 0.0551 - val_accuracy: 0.9824
Epoch 2/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0464 - accuracy: 0.9858 - val_loss: 0.0371 - val_accuracy: 0.9872
Epoch 3/12
469/469 [==============================] - 7s 16ms/step - loss: 0.0290 - accuracy: 0.9910 - val_loss: 0.0350 - val_accuracy: 0.9873
Epoch 4/12
469/469 [==============================] - 8s 17ms/step - loss: 0.0198 - accuracy: 0.9938 - val_loss: 0.0403 - val_accuracy: 0.9875
Epoch 5/12
469/469 [==============================] - 7s 14ms/step - loss: 0.0138 - accuracy: 0.9956 - val_loss: 0.0406 - val_accuracy: 0.9876
Epoch 6/12
469/469 [==============================] - 7s 14ms/step - loss: 0.0098 - accuracy: 0.9969 - val_loss: 0.0351 - val_accuracy: 0.9903
Epoch 7/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0091 - accuracy: 0.9969 - val_loss: 0.0395 - val_accuracy: 0.9894
Epoch 8/12